In [ ]:
%pip install --upgrade langchain langchain-experimental langchain-openai python-dotenv pyvis

In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()
# Get API key from environment variable 
# (make sure the key is present in .env file in the project directory)
api_key = os.getenv("OPENAI_API_KEY")

### LLM Graph Transformer
Using GPT-4o in all examples.

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

graph_transformer = LLMGraphTransformer(llm=llm)

### Extract graph data

In [ ]:
text = """
Luật 96/2025/QH15 các tổ chức tín dụng sửa đổi năm 2025:

Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam đã được sửa đổi, bổ sung một số điều theo Nghị quyết số 203/2025/QH15;
"""

In [ ]:
documents = [Document(page_content=text)]

In [ ]:
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

#### Visualize graph

In [ ]:
from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")

In [ ]:
# Run the function
visualize_graph(graph_documents)

### Extract specific types of nodes

In [ ]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{graph_documents_nodes_defined[0].nodes}")
print(f"Relationships:{graph_documents_nodes_defined[0].relationships}")

### Extract specific types of relationships

In [ ]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
graph_transformer_rel_defined = LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships
)
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [ ]:
# Visualize graph
visualize_graph(graph_documents_rel_defined)

### Extract Vietnamese legal entities with custom prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Custom prompt for Vietnamese legal document graph extraction.
VIETNAMESE_LEGAL_SYSTEM_PROMPT = (
    "Bạn là một chuyên gia phân tích văn bản pháp luật Việt Nam. " \
    "Nhiệm vụ của bạn là trích xuất một đồ thị tri thức từ văn bản lập pháp tiếng Việt.\n\n" \
    "Ví dụ: " \
    "Luật 96/2025/QH15 các tổ chức tín dụng sửa đổi năm 2025:\n" \
    "Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam đã được sửa đổi, bổ sung một số điều theo Nghị quyết số 203/2025/QH15;\n\n" \
    "Nodes: ['Luật 96/2025/QH15 các tổ chức tín dụng sửa đổi năm 2025', 'Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam', 'Nghị quyết số 203/2025/QH15']\n" \
    "Relationships: [('Luật 96/2025/QH15 các tổ chức tín dụng sửa đổi năm 2025', 'CAN_CU', 'Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam'), ('Nghị quyết số 203/2025/QH15', 'SUA_DOI_BO_SUNG', 'Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam')]" \
)

custom_legal_prompt = ChatPromptTemplate.from_messages([
    ("system", VIETNAMESE_LEGAL_SYSTEM_PROMPT),
    ("human", "{input}"),
])

# Build transformer with domain constraints + custom prompt
allowed_nodes_legal = ["Luật", "Nghị quyết", "Hiến pháp", "Điều khoản"]
allowed_relationships_legal = [
    "CAN_CU",
    "SUA_DOI_BO_SUNG",
    "BAI_BO",
    "DAN_CHIEU",
    "HUY_BO",
    "HOP_NHAT",
    "THAY_THE",
    "QUY_DINH_CHI_TIET",
    "KEO_DAI_HIEU_LUC",
    "DINH_CHI",
    "DINH_CHINH",
    "HUONG_DAN"
]

graph_transformer_legal = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=allowed_nodes_legal,
    allowed_relationships=allowed_relationships_legal,
    prompt=custom_legal_prompt,
    strict_mode=False,
)

graph_documents_legal = await graph_transformer_legal.aconvert_to_graph_documents(documents)
print(f"Nodes: {graph_documents_legal[0].nodes}")
print(f"\nRelationships: {graph_documents_legal[0].relationships}")

In [ ]:
def normalize_node_casing(graph_docs):
    """
    LLMGraphTransformer title-cases node IDs internally.
    This function normalizes all node IDs to lowercase.
    """
    for graph_doc in graph_docs:
        for node in graph_doc.nodes:
            node.id = node.id.lower()
        for rel in graph_doc.relationships:
            rel.source.id = rel.source.id.lower()
            rel.target.id = rel.target.id.lower()
    return graph_docs


graph_documents_legal = normalize_node_casing(graph_documents_legal)
print(f"Nodes: {graph_documents_legal[0].nodes}")
print(f"\nRelationships: {graph_documents_legal[0].relationships}")

In [ ]:
# Visualize the legal knowledge graph
visualize_graph(graph_documents_legal)